In [ ]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from openai import AsyncOpenAI

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [ ]:
class SubTask(BaseModel):
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


class TravelPlan(BaseModel):
    main_task: str = Field(
        description="The overall travel request from the user")
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

In [ ]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = OpenAIChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

In [ ]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

# Mahahalagang Panuntunan

Ang dokumentong ito ay naglalaman ng mga mahahalagang impormasyon na dapat sundin upang masiguro ang tamang paggamit ng aming produkto. Mangyaring basahin nang mabuti ang bawat seksyon.

## Pangkalahatang Impormasyon

Ang aming produkto ay dinisenyo upang maging madaling gamitin at maaasahan. Gayunpaman, mahalaga pa rin na sundin ang mga alituntunin upang maiwasan ang anumang problema.

[!NOTE] Ang mga detalyeng ibinigay dito ay maaaring magbago nang walang paunang abiso.

## Mga Kinakailangan

Bago gamitin ang produkto, tiyaking natutugunan ang mga sumusunod na kinakailangan:

1. Ang iyong sistema ay dapat mayroong pinakabagong bersyon ng operating system.
2. Siguraduhing naka-install ang lahat ng kinakailangang dependencies.
3. Basahin ang dokumentasyon para sa karagdagang impormasyon.

[!WARNING] Ang hindi pagsunod sa mga kinakailangan ay maaaring magdulot ng hindi inaasahang resulta.

## Mga Hakbang sa Pag-install

Sundin ang mga hakbang na ito upang maayos na mai-install ang produkto:

1. I-download ang installer mula sa opisyal na website.
2. Buksan ang installer at sundin ang mga tagubilin sa screen.
3. Kapag natapos na ang pag-install, i-restart ang iyong sistema.

[!TIP] Para sa mas mabilis na pag-install, tiyaking walang ibang tumatakbong application sa iyong sistema.

## Pag-troubleshoot

Kung makakaranas ng anumang problema, subukan ang mga sumusunod na hakbang:

1. Suriin ang log files para sa anumang error messages.
2. Tiyaking ang lahat ng dependencies ay naka-install nang maayos.
3. Kung hindi pa rin maayos ang problema, makipag-ugnayan sa aming support team.

[!IMPORTANT] Mangyaring huwag baguhin ang anumang configuration files maliban kung sigurado ka sa iyong ginagawa.

## Feedback

Malugod naming tinatanggap ang iyong mga puna at mungkahi. Mangyaring makipag-ugnayan sa amin sa pamamagitan ng aming opisyal na website o email.

Salamat sa paggamit ng aming produkto!



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
